In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as F


spark = SparkSession \
    .builder \
    .config("spark.master", "cluster") \ # Определяет режим запуска приложения (локальный, standalone, YARN, Mesos).
    .config("spark.driver.memory", '2g') \ # Определяет объем памяти, выделенный для драйвера.
    .config("spark.executor.instances", 2) \ # Определяет количество исполнителей (executor).
    .config("spark.executor.cores", '2') \  # Определяет количество ядер, выделяемых для каждого исполнителя.
    .config("spark.executor.memory", '4g') \ # Определяет объем памяти, выделенный для каждого исполнителя.
    .appName("halltape_pyspark_local") \ # это будет написано в YARN
    .getOrCreate()


print("Активные Spark сессии:", spark.sparkContext.uiWebUrl)

In [ ]:
from pyspark.sql import SparkSession

# Создаем SparkSession с настройками
spark = SparkSession.builder \
    .appName("example_app") \  # Имя приложения
    .master("yarn") \  # Режим запуска (YARN)
    
    # 1. Конфигурационные параметры Spark
    .config("spark.driver.memory", "2g") \  # Объем памяти для драйвера (2 гигабайта)
    .config("spark.executor.memory", "4g") \  # Объем памяти для каждого исполнителя (4 гигабайта)
    .config("spark.executor.instances", "2") \  # Количество исполнителей (2)
    .config("spark.default.parallelism", "200") \  # Количество партиций по умолчанию (200)
    
    # 2. Параметры управления ресурсами
    .config("spark.cores.max", "4") \  # Максимальное количество ядер для приложения (4)
    .config("spark.executor.cores", "2") \  # Количество ядер на исполнителя (2)
    .config("spark.dynamicAllocation.enabled", "true") \  # Включение динамического выделения ресурсов
    
    # 3. Параметры безопасности
    .config("spark.authenticate", "true") \  # Включение аутентификации между компонентами
    .config("spark.ssl.enabled", "true") \  # Включение SSL/TLS для защищенного соединения
    
    # 4. Параметры кэширования и хранения данных
    .config("spark.local.dir", "/tmp/spark") \  # Путь для временных данных (shuffle, кэш)
    .config("spark.storage.memoryFraction", "0.6") \  # Доля памяти для кэширования данных (60%)
    
    # 5. Параметры уровня журналирования и мониторинга
    .config("spark.eventLog.enabled", "true") \  # Включение журналирования событий
    .config("spark.executor.logs.rolling.maxRetainedFiles", "5") \  # Максимальное количество файлов журналов (5)
    
    .getOrCreate()  # Создание или получение существующей SparkSession

# Проверка конфигурации
print("Текущие настройки Spark:")
print(spark.sparkContext.getConf().getAll())

# Пример работы с данными
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Вывод данных
df.show()

# Остановка SparkSession
spark.stop()

In [ ]:
'''YARN это ресурс менеджер который разбивает и находит все ресурсы, говорит нам где и что у нас запущенно

Driver нужен для того чтобы покозать инф с executor.. собирает инф с них . Он может находиться или отдельно на компе вне кластера или прям на сластаре
драйвер управляет и собирает'''


In [ ]:
ls  # какие есть папки 

In [ ]:
ls dataset/ # это папка  открыть там три таблички

In [ ]:
pwd # покажет путь  

In [ ]:
PATH = 'ТУТ ПОЛНЫЙ ПУТЬ '

card = spark.read.csv(f"{PATH}/card.csv",header=True, sep=';') # header что есть название колонок ,  sep= разделитель
card = spark.read.csv("тут путь card.csv ИЛИ ПОЛНЫЙ ПУТЬ ЕСЛИ НЕ ПРОЧИТАЕТ")
date = '2020-06-01'
card = spark.read.csv(f"{PATH}/card.csv",header=True, sep=';').where('''load_date="{date}"''')

In [ ]:
PATH = 'ТУТ ПОЛНЫЙ ПУТЬ '
date = '2020-06-01'
'''ТУТ ТРИ ТАБЛИЦЫ'''
card = spark.read.csv(f"{PATH}/card.csv",header=True, sep=';').where('''load_date="{date}"''') 
status = spark.read.csv(f"{PATH}/status.csv",header=True, sep=';').where('''load_date="{date}"''')
trx = spark.read.csv(f"{PATH}/trx.csv",header=True, sep=';').where('''load_date="{date}"''')

ВИТИРИНА ДАННЫХ
ЗАДАЧА СЖОЙНИТЬ ЭТИ ТРИ ТАБЛИЦЫ И ВЫВЕСТИ НУЖЫНЕ ДАННЫЕ
И ТУТ ПОЛЯ 

In [ ]:
trx_df = trx.groupBy('card_num').agg(F.min('amount').alias('amount')).where('''card_num = "345678"''')
trx_df.show()
# мы тут сгруппировали стобец card_num где, столбец amount самый min.. 
# и вывели только конкрентную карту 345678 в стобце card_num 

In [ ]:
trx_df.columns

In [ ]:
trx_df_p = trx_df.join(trx, ['card_num', 'transaction_datetime'], 'inner').columns
# если мы просто хотим посмотреть какие колонки вызовутся при джоине, то саму оперцию не вызовет и ресурсы жрать не будет

In [ ]:
# джойним наш датафрейм trx_df с второй таблицей status
df_2 = status.join(trx_df, 'card_num', 'inner').show() 

In [ ]:
# джойним третию таблицу 
df_2.join(card, 'card_num_md5', 'inner').show() # ТУТ ОНА БУДЕТ СЛИШКОМ БОЛЬШАЯ И ПО ЭТОМУ МЫ ПИШМ ТАК 
result = df_2(()).join(card, 'card_num_md5', 'inner').show(5,False,True) # он перевернет в столбики слево и будет удобно смотреть